# SageMaker Inference from Scratch

KLUE RoBERTa 모델을 SageMaker Endpoint로 배포하고 추론합니다.

## 0. 로컬 추론 함수 테스트

먼저 로컬에서 inference.py가 정상 동작하는지 확인합니다.

In [1]:
! which python
! uv pip list | grep torch

/home/ubuntu/lab/16-robert-sagemaker-inference/.venv/bin/python
Using Python 3.11.0rc1 environment at: /home/ubuntu/lab/16-robert-sagemaker-inference/.venv
torch                            2.5.0


In [2]:
!python ../src/test_inference.py

Testing SageMaker Inference Functions

1. Testing model_fn...
Loading model from .
Using device: cuda
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Model loaded successfully
   ✓ Model loaded

2. Testing input_fn...
Received content_type: application/json
   ✓ Processed 3 texts

3. Testing predict_fn...
Starting prediction
Processing 3 text(s)
Prediction completed successfully
   ✓ Embedding shape: (3, 768)

4. Testing output_fn...
Formatting output with accept: application/json
   ✓ Output keys: ['embeddings', 'embedding_dim', 'num_texts']

All tests passed!


## 1. 환경 설정

In [3]:
import json
import time
import boto3
import sagemaker
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

sagemaker_session = sagemaker.Session()
role = "arn:aws:iam::057716757052:role/gonsoomoon-sm-inference"
bucket = sagemaker_session.default_bucket()

print(f"Bucket: {bucket}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml
Bucket: sagemaker-us-east-1-057716757052


## 2. 모델 아티팩트 생성 및 S3 업로드

model.tar.gz 구조로 생성을 하면 , SageMaker 가 이를 인지 합니다.
model.tar.gz 구조:
```
model.tar.gz/
├── config.json
├── model.safetensors
├── special_tokens_map.json
├── tokenizer.json
├── tokenizer_config.json
├── vocab.txt
└── code/
    ├── inference.py
    └── requirements.txt
```

참조: [SageMaker PyTorch Documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#deploy-pytorch-models)

### model.tar.gz 파일 생성

In [4]:
!rm -rf ../model_artifact
!mkdir -p ../model_artifact/code
!cp ../src/inference.py ../model_artifact/code/
!cp ../src/requirements.txt ../model_artifact/code/
!cp ../model/* ../model_artifact/

!cd ../model_artifact && tar -czf ../model.tar.gz *

model_artifact_s3_uri = f's3://{bucket}/klue-roberta-inference/model/model.tar.gz'
!aws s3 cp ../model.tar.gz {model_artifact_s3_uri}

print(f"Model uploaded to: {model_artifact_s3_uri}")

upload: ../model.tar.gz to s3://sagemaker-us-east-1-057716757052/klue-roberta-inference/model/model.tar.gz
Model uploaded to: s3://sagemaker-us-east-1-057716757052/klue-roberta-inference/model/model.tar.gz


## 3. SageMaker Endpoint 생성

In [5]:
import os

model_data_dir = ".."
local_model_path = os.path.join(model_data_dir, 'model.tar.gz')
print("local_model_path: ", local_model_path)

local_model_path:  ../model.tar.gz


In [6]:
import os
import subprocess

try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
    else:
        instance_type = "local"        
except:
    pass

print("Instance type = " + instance_type)

Sat Sep 27 04:29:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.20             Driver Version: 570.133.20     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:36:00.0 Off |                    0 |
| N/A   42C    P8             16W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Instance type = local_gpu


In [7]:
endpoint_name = "local-endpoint-ncf-{}".format(int(time.time()))

local_pytorch_model = PyTorchModel(model_data=local_model_path,
                                   role=role,
                                   entry_point='inference.py',
                                   source_dir = '../src',
                                   framework_version='2.5',
                                   py_version='py311',
                                   model_server_workers=1,
                                  )

local_predictor = local_pytorch_model.deploy(
                           instance_type=instance_type, 
                           initial_instance_count=1, 
                           endpoint_name=endpoint_name,
                           wait=True,
                           log = False,
                        )

RootlessDocker not detected, falling back to remote host IP or localhost.


Attaching to f3li42g0j0-algo-1-0f1av
f3li42g0j0-algo-1-0f1av  | CUDA compat package should be installed for NVIDIA driver smaller than 550.163.01
f3li42g0j0-algo-1-0f1av  | Current installed NVIDIA driver version is 570.133.20
f3li42g0j0-algo-1-0f1av  | Skipping CUDA compat setup as newer NVIDIA driver is installed
f3li42g0j0-algo-1-0f1av  | /opt/conda/lib/python3.11/site-packages/sagemaker_pytorch_serving_container/torchserve.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
f3li42g0j0-algo-1-0f1av  |   import pkg_resources
f3li42g0j0-algo-1-0f1av  | Collecting transformers>=4.30.0 (from -r /opt/ml/model/code/requirements.txt (line 1))
f3li42g0j0-algo-1-0f1av  |   Downloading transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
f3li42g0j0-algo-1-0f1av  | Requirement already satisfied: fil

## 이전 코드

## 4. Endpoint 추론

In [8]:
local_predictor.serializer = JSONSerializer()
local_predictor.deserializer = JSONDeserializer()

result = local_predictor.predict({"texts": "김치찌개와 된장찌개는 한국의 대표 음식입니다."})



RootlessDocker not detected, falling back to remote host IP or localhost.


f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:48,837 [INFO ] epollEventLoopGroup-3-2 TS_METRICS - ts_inference_requests_total.Count:1.0|#model_name:model,model_version:default|#hostname:27c38f57aa3a,timestamp:1758947568
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:48,838 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Looping backend response at: 1758947568838
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:48,839 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Backend received inference at: 1758947568
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:48,839 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Received content_type: application/json
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:48,839 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Received content_type: application/json
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:48,839 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Starting prediction
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:48,840 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Starting 

f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,063 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Prediction completed successfully
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,064 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Prediction completed successfully
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,064 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Formatting output with accept: application/json
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,064 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Formatting output with accept: application/json
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,064 [INFO ] W-9000-model_1.0-stdout org.pytorch.serve.wlm.WorkerLifeCycle - result=[METRICS]PredictionTime.Milliseconds:225.18|#ModelName:model,Level:Model|#type:GAUGE|#hostname:27c38f57aa3a,1758947569,1c5b321e-d315-407c-9510-f75ba40755ea, pattern=[METRICS]
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,065 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.BatchAggregator - Sending response for jobId 1c5b321e

In [9]:
import numpy as np

def print_result(result):
    embedding_dim = result["embedding_dim"]
    num_texts = result["num_texts"]

    print("shape: ", np.shape(result["embeddings"]))
    print(f"embedding_dim: {embedding_dim}")
    print("num_texts: ", num_texts)

print_result(result)


shape:  (1, 768)
embedding_dim: 768
num_texts:  1


In [10]:
import numpy as np
import time

def test_similarity(predictor, query, documents, top_k=3):
    """
    쿼리와 문서들 간의 유사도를 계산하고 결과를 출력합니다.
    
    Args:
        predictor: SageMaker predictor 객체
        query (str): 검색 쿼리
        documents (list): 비교할 문서들의 리스트
        top_k (int): 상위 몇 개의 문서를 출력할지 (기본값: 3)
    """
    def cosine_similarity(query_emb, doc_embs):
        """코사인 유사도 계산"""
        query_emb = np.array(query_emb)
        doc_embs = np.array(doc_embs)
        
        # 정규화
        query_norm = query_emb / np.linalg.norm(query_emb, axis=-1, keepdims=True)
        doc_norms = doc_embs / np.linalg.norm(doc_embs, axis=-1, keepdims=True)
        
        # 코사인 유사도
        return np.dot(query_norm, doc_norms.T).squeeze(0)
    
    # 헤더 출력
    print("=" * 70)
    print(f"🔍 쿼리: '{query}'")
    print("=" * 70)
    
    # 임베딩 변환
    start = time.time()
    query_result = predictor.predict({"texts": query})
    query_emb = query_result["embeddings"]
    
    docs_result = predictor.predict({"texts": documents})
    doc_embs = docs_result["embeddings"]
    encode_time = (time.time() - start) * 1000
    
    print(f"\n✅ 임베딩 완료")
    print(f"   - 쿼리 임베딩: {np.shape(query_emb)}")
    print(f"   - 문서 임베딩: {np.shape(doc_embs)}")
    print(f"   - 인코딩 시간: {encode_time:.1f}ms")
    
    # 유사도 계산
    similarities = cosine_similarity(query_emb, doc_embs)
    
    # 전체 결과 출력
    print("\n" + "=" * 70)
    print("📊 유사도 결과:")
    print("-" * 70)
    
    for idx, (doc, score) in enumerate(zip(documents, similarities), 1):
        bar = "█" * int(score * 30)
        print(f"\n{idx}. [{score:.4f}] {bar}")
        print(f"   {doc}")
    
    # 상위 K개 문서 추출
    print("\n" + "=" * 70)
    print(f"🏆 상위 {top_k}개 문서:")
    print("-" * 70)
    
    top_k_indices = np.argsort(similarities)[::-1][:top_k]
    for rank, idx in enumerate(top_k_indices, 1):
        score = similarities[idx]
        print(f"\n{rank}위. [{score:.4f}] {documents[idx]}")
    
    print("=" * 70)

In [11]:
# 테스트 데이터 정의
query = "맛있는 한국 음식"

documents = [
    "김치찌개와 된장찌개는 한국의 대표 음식입니다.",        # 매우 관련
    "서울 강남의 유명한 한식당을 소개합니다.",             # 관련
    "일본 라멘과 초밥은 인기가 많습니다.",                # 약간 관련 (음식)
    "파리의 에펠탑은 프랑스의 상징입니다.",               # 무관 (관광)
    "삼성전자 주가가 상승했습니다.",                     # 완전 무관 (경제)
    "양자 컴퓨터의 발전이 가속화되고 있습니다.",          # 완전 무관 (기술)
]

# 유사도 테스트 실행
test_similarity(local_predictor, query, documents, top_k=3)

RootlessDocker not detected, falling back to remote host IP or localhost.
RootlessDocker not detected, falling back to remote host IP or localhost.


🔍 쿼리: '맛있는 한국 음식'
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,150 [INFO ] epollEventLoopGroup-3-2 TS_METRICS - ts_inference_requests_total.Count:1.0|#model_name:model,model_version:default|#hostname:27c38f57aa3a,timestamp:1758947569
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,150 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Looping backend response at: 1758947569150
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,151 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Backend received inference at: 1758947569
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,152 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Received content_type: application/json
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,152 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Received content_type: application/json
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,152 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Starting prediction
f3li42g0j0-algo-1-0f1av  | 2025-09-27T04:32:49,152 [INFO ] W-9000-model_1.0-stdout MOD

## 엔드포인트 제거

In [12]:
local_predictor.delete_endpoint(delete_endpoint_config=True)
print("✅ Endpoint 삭제 완료")

Gracefully Stopping... press Ctrl+C again to force


✅ Endpoint 삭제 완료
